<a href="https://colab.research.google.com/github/nabiharaza/-Machine_Learning_ExpertSystem-/blob/master/Accident_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSCI-639 Accident Prediction
#### **Authors-  Nabiha Raza, Neel Kirit**

## Papers Referred - 
* Blah Blah

## Design Overview -
* Some shit

## Model overview -
* Fouck you

<hr>

## Step 1
We import all necceary libs we have used in our Project

In [ ]:
#https://www.notion.so/Crash-1500-bd45c6772b10436592241544b027d356
#https://drive.google.com/drive/folders/0APfpY9WB1JnvUk9PVA


# Import all the libs 
import numpy as np
import os
import re
import pickle
import timeit
import glob
import cv2

from skimage import transform
import skimage
from skimage import io

import sklearn
from sklearn.model_selection import train_test_split   ### import sklearn tool

import keras
from keras.preprocessing import image as image_utils
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, TimeDistributed
from skimage import io

## Step 2
Mount Google Drive to load 1500 videos for training / tesing 

**For Grader** <br>
Please click on the URL below and authorize your google drive, then point to the directory where you have kept 1500 files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 3
Load the dataset by reading the videos and extracting the frames

In [ ]:
def load_dataset(path_to_video, total_frames):
    # Capture a video
    vidcap = cv2.VideoCapture(path_to_video)
    # Check if video frame is read correctly
    success,image = vidcap.read()
    count = 0
    img = []
    # Read all frames of video iteratively
    while success:    
        success,image = vidcap.read()
        tmp = skimage.color.rgb2gray(np.array(image))
        img.append(tmp)
        count += 1
        if count == total_frames:
            break
    return img

## Step 4
Read the `1500.txt` file and start loading the dataset
Spit out the # of videos that will be used out for this project

In [ ]:
data = []
labels = []
all_frames = []
f = open("Annotations.txt", "r")
lines = f.readlines()
for line in lines[1:10]:
    video_name = line[0:6]
    y_labels = line[10:156].split(',')
    y_labels = [int(i[1]) for i in y_labels ]
    total_frames = len(y_labels)
    
    x1 = line[159:].split(',')[2]
    x2 = line[159:].split(',')[3]
    is_ego = line[159:].split(',')[4]
    if "Yes" in is_ego:
        frame_count = 0
        for i in y_labels:
          frame_count += 1
          if i == 1:
              break
        flag = 0
        for i in y_labels:
            if i == 1 and flag == 0:

              break
        for i in range(len(y_labels)):
          if y_labels[i] != 1:
              y_labels[i] = frame_count
              frame_count -= 1
          else:
              break
                
        labels.extend(y_labels)
        img = load_dataset("Crash1500/"+video_name+".mp4", total_frames)
        all_frames.extend(img)
# Check if all videos' frame are loaded in np array
print("# of videos loaded",len(all_frames))
print("# of labels", len(labels))

## Step 5
Split the data into train and test

In [0]:
# Split into train and test sets
# all_frames = np.array(all_frames)
x_train, x_test, y_train, y_test = train_test_split(all_frames, labels, test_size=0.40, random_state=0)  ### split
# print(x_train)

print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

r = np.array(x_train[0])
g = np.array(x_train[1])
b = np.array(x_train[2])
x_train = np.dstack((r,g,b))
print(x_train.shape)

r = np.array(x_test[0])
g = np.array(x_test[1])
b = np.array(x_test[2])
x_test = np.dstack((r,g,b))
y_train = np.array(y_train)
y_test = np.array(y_test)
print(y_train.shape)

In [0]:
### set hyper-parameters
batch_size = 15
num_classes = 2
epochs = 30

### number of hidden layers in each NN
row_hidden = 128
col_hidden = 128

### get shape of rows/columns for each image
frame, row, col = (9, 720, 1280)

### 4D input - for each 3-D sequence (of 2-D image) in each video (4th)
x = Input(shape=(frame, row, col))

encoded_rows = TimeDistributed(LSTM(row_hidden))(x)  ### encodes row of pixels using TimeDistributed Wrapper
encoded_columns = LSTM(col_hidden)(encoded_rows)     ### encodes columns of encoded rows using previous layer

### set up prediction and compile the model
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy', ### loss choice for category classification - computes probability error
              optimizer='NAdam',               ### NAdam optimization
              metrics=['accuracy'])    
# model.fit(x_train, y_train,                            ### fit training data
#                   batch_size=len(x_train),                ### reiterate batch size - in this case we already set up the batches
#                   epochs=100,                          ### number of times to run through each batch
#                   validation_data=(x_test, y_test))  



# model = Sequential()
# model.add(LSTM(4, input_shape=(1280, 1)))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=1)

TypeError: ignored